In [1]:
import numpy as np

### Lecture des donnees

In [2]:
file_train = open("cascades_train.txt")
file_test = open("cascades_test.txt")


data_train=[]
for i in file_train.readlines():
    tab = [[float(j.split(":")[0]),float(j.split(":")[1])] for j in i.split(";")[:-1]]
    tab.sort(key=lambda x: x[1])
    data_train.append(tab)

    
data_test=[]
for i in file_test.readlines():
    tab = [[float(j.split(":")[0]),float(j.split(":")[1])] for j in i.split(";")[:-1]]
    tab.sort(key=lambda x: x[1])
    data_test.append(tab)

### Creation des liste de successeurs

In [3]:
def getSuccsOfTab(tab):
    succ=[]
    for i in tab:
        succ_i=[]
        for j in tab:
            if(i[1]<j[1]):
                succ_i.append(j[0])
        succ.append([i[0],succ_i])
    return succ

def unique(liste):
    seen=set()
    seen_add =seen.add
    return [x for x in liste if not(x in seen or seen_add(x))]

In [4]:
succs_train = []
for i in data_train:
    succs_train.append(getSuccsOfTab(i))

succs_test = []
for i in data_test:
    succs_test.append(getSuccsOfTab(i))

## Création des graph

In [5]:
#optimisation au niveau du temps possible
def getGraph(succs):
    graph={}
    for h in succs:
        for i in h:
            try:
                for j in i[1]:
                    graph[i[0]].append(j)
                graph[i[0]]=unique(graph[i[0]])
                graph[i[0]].sort()
            except KeyError:
                graph[i[0]]=i[1]
    return graph

In [6]:
graph_train = getGraph(succs_train)
graph_test = getGraph(succs_test)

## Apprentissage

#### Fonction pour avoir la probabilité, P chapeau

In [7]:
#liste correspond a un D
def getListPrec(liste):
    prec=[]
    for i in liste:
        if (i[1]>1):
            p=[]
            for j in liste:
                if (i[1]>j[1]):
                    p.append(j[0])
            prec.append((i[0],p))
    return prec

In [8]:
#listePrec obtenue avec getListPrec dun D
def getProbaOfList(listePrec,graph_weight_d):
    a=[]
    for i in listePrec:
        prod=1
        for pre in i[1]:
            prod= prod *(1 - graph_weight_d[pre][i[0]])
        a.append((i[0],1-prod))
    return a

### Poids du graphe assignés en random

In [20]:
#PAS UTILE
#creation des poids du graph
def getGraphou(graph):
    graph_weight={}
    for i in graph:
        for e in graph[i]:
            rando=np.random.rand()
            try:
                graph_weight[i].append((e,rando))
            except KeyError:
                graph_weight[i]=[(e,rando)]

In [12]:
#creation des poids du graph dicodico
def getGraphWeightRandom(graph):
    graph_weight_d={}
    for i in graph:
        dico={}
        for e in graph[i]:
            rando=np.random.rand()
            dico[e]=rando
        graph_weight_d[i]=dico
    return graph_weight_d

In [13]:
gInit_train = getGraphWeightRandom(graph_train)

#### Fonction d'apprentissage

In [17]:
#a enlever graph_train de la signature
def fitModele(graph_train,data_train,nbIt=10,eps=1e-1):
    graph_cur = getGraphWeightRandom(graph_train)
    it=0
    ep=100000
    while(it<nbIt & ep>eps):
        probas_chapiteau=[getProbaOfList(getListPrec(d),graph_cur) for d in data_train]
        
        # mettre a jour les poids
        
        # calculer la difference entre poids de l'ancien graphe et le nouveau
        # ancien graph=nouveau
        it+=1

----

# BROUILLON

In [18]:
probas_chapiteau=[getProbaOfList(getListPrec(d),gInit_train) for d in data_train]

In [19]:
probas_chapiteau[0]

[(42.0, 0.980359892005104),
 (41.0, 0.8802982756212641),
 (93.0, 0.3123363527127121),
 (89.0, 0.9859272987687169),
 (8.0, 0.9999177565628776),
 (3.0, 0.9908071159837314),
 (43.0, 0.9999485737361878),
 (73.0, 0.9993305471621168),
 (84.0, 0.9999994985634716),
 (61.0, 0.9999957612491343),
 (98.0, 0.9996427014821843),
 (86.0, 0.9999988678463826),
 (48.0, 0.9999698744389857),
 (47.0, 0.9994914261562857),
 (96.0, 0.9999999997126455),
 (50.0, 0.9999701830450044),
 (55.0, 0.9999999999918294),
 (20.0, 0.9999999932706135),
 (12.0, 0.9999952154811971),
 (5.0, 0.9999999978175025),
 (52.0, 0.9999999970348916),
 (54.0, 0.9999999781037353),
 (34.0, 0.9999999999898997),
 (4.0, 0.9999999999972279),
 (82.0, 0.9999999999985689)]

In [65]:
import re


texte ="<tr><td class='Ligne'>4I200--2014oct</td><td class='Ligne'>[examen-reparti-1] examen-reparti-1</td><td  class='Ligne' align='right'><strong>11</strong>/20</tr><tr><td class='Ligne'>4I200--2014oct</td><td class='Ligne'>[examen] examen reparti 2</td><td  class='Ligne' align='right'><strong>8</strong>/20</tr><tr><td class='Ligne'>4I200--2014oct</td><td class='Ligne'>[epreuve] Evaluation projet optimisation equitable</td><td  class='Ligne' align='right'><strong>16</strong>/20</tr><tr><td class='Ligne'>4I502--2014oct</td><td class='Ligne'>[examen] examen réparti 1</td><td  class='Ligne' align='right'><strong>11</strong>/20</tr><tr><td class='Ligne'>4I502--2014oct</td><td class='Ligne'>[cc-total] Projet LDVH</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>4I502--2014oct</td><td class='Ligne'>[epreuve] Examen Réparti 2 : Janvier</td><td  class='Ligne' align='right'><strong>10</strong>/20</tr><tr><td class='Ligne'>4I601--2014oct</td><td class='Ligne'>[epreuve] exam de janvier</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>4I601--2014oct</td><td class='Ligne'>[epreuve] CC</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>4I601--2014oct</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 30-01-2015-533163</td><td  class='Ligne' align='right'><strong>14.3</strong>/20</tr><tr><td class='Ligne'>4I601--2014oct</td><td class='Ligne'>[examen] note dbufr 1ere session</td><td  class='Ligne' align='right'><strong>14.3</strong>/20</tr><tr><td class='Ligne'>4I800--2014oct</td><td class='Ligne'>[examen-reparti-2] creation du 19-01-2015-533162</td><td  class='Ligne' align='right'><strong>7.5</strong>/20</tr><tr><td class='Ligne'>4I800--2014oct</td><td class='Ligne'>[examen] creation du 06-12-2014-112</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>4I801--2014oct</td><td class='Ligne'>[examen-reparti-1] 5 novembre 2014</td><td  class='Ligne' align='right'><strong>17</strong>/20</tr><tr><td class='Ligne'>4I801--2014oct</td><td class='Ligne'>[examen-reparti-2] janvier 2015</td><td  class='Ligne' align='right'><strong>16</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[epreuve] projet 1</td><td  class='Ligne' align='right'><strong>15</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[epreuve] Projet 2</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[examen-reparti-2] examen</td><td  class='Ligne' align='right'><strong>11</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 03-06-2015-533163</td><td  class='Ligne' align='right'><strong>12.4</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[examen-reparti-1] creation du 03-06-2015-533161</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>4I803--2015fev</td><td class='Ligne'>[examen-reparti-1] examen-reparti-1</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>4I803--2015fev</td><td class='Ligne'>[examen-reparti-2] 19 mai 2015</td><td  class='Ligne' align='right'><strong>12</strong>/20</tr><tr><td class='Ligne'>4I803--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] note finale session1</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>4I804--2015fev</td><td class='Ligne'>[examen] creation du 05-06-2015-112</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>4I804--2015fev</td><td class='Ligne'>[examen-reparti-1] creation du 05-06-2015-533161</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>4I804--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 05-06-2015-533163</td><td  class='Ligne' align='right'><strong>13.4</strong>/20</tr><tr><td class='Ligne'>4I807--2015fev</td><td class='Ligne'>[epreuve] Recherche Bibliographique</td><td  class='Ligne' align='right'><strong>18</strong>/20</tr><tr><td class='Ligne'>4I807--2015fev</td><td class='Ligne'>[epreuve] Rapport + Soutenance</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>4I807--2015fev</td><td class='Ligne'>[epreuve] Projet</td><td  class='Ligne' align='right'><strong>12</strong>/20</tr><tr><td class='Ligne'>4I807--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 04-06-2015-533163</td><td  class='Ligne' align='right'><strong>12.65</strong>/20</tr><tr><td class='Ligne'>4I808--2015fev</td><td class='Ligne'>[examen-reparti-1] Projet (rapport, soutenance, application)</td><td  class='Ligne' align='right'><strong>12.5</strong>/20</tr><tr><td class='Ligne'>4I808--2015fev</td><td class='Ligne'>[examen-reparti-2] Examen de fin de semestre</td><td  class='Ligne' align='right'><strong>16.45</strong>/20</tr><tr><td class='Ligne'>4I808--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 02-06-2015-533163</td><td  class='Ligne' align='right'><strong>14.08</strong>/20</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[TME solitaire] creation du 10-02-2012-477</td><td  class='Ligne' align='right'><strong>18.75</strong>/20</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[TD] creation du 10-02-2012-2648</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[partiel] creation du 10-02-2012-108</td><td  class='Ligne' align='right'><strong>8</strong>/20</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[examen] creation du 10-02-2012-112</td><td  class='Ligne' align='right'><strong>40.75</strong>/70</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[cc-total] creation du 10-02-2012-111</td><td  class='Ligne' align='right'><strong>12.4</strong>/20</tr><tr><td class='Ligne'>LI213-2013fev</td><td class='Ligne'>[epreuve] Epreuve écrite 1ere session</td><td  class='Ligne' align='right'><strong>13.25</strong>/20</tr><tr><td class='Ligne'>LI213-2013fev</td><td class='Ligne'>[Epreuve TP] TME1-2</td><td  class='Ligne' align='right'><strong>8</strong>/20</tr><tr><td class='Ligne'>LI213-2013fev</td><td class='Ligne'>[TD] Note TD (interros+participation)</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>LI213-2013fev</td><td class='Ligne'>[Epreuve TP] Projet</td><td  class='Ligne' align='right'><strong>15</strong>/20</tr><tr><td class='Ligne'>LI215-2012oct</td><td class='Ligne'>[participation] TP et Participation</td><td  class='Ligne' align='right'><strong>16.75</strong>/20</tr><tr><td class='Ligne'>LI215-2012oct</td><td class='Ligne'>[examen] Ecrit première Session</td><td  class='Ligne' align='right'><strong>19.5</strong>/60</tr><tr><td class='Ligne'>LI215-2012oct</td><td class='Ligne'>[cc-total] creation du 16-09-2012-111</td><td  class='Ligne' align='right'><strong>24.25</strong>/40</tr><tr><td class='Ligne'>LI215-2012oct</td><td class='Ligne'>[partiel] Partiel barême ajusté</td><td  class='Ligne' align='right'><strong>7.5</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir sur table] 2ième interrogation de TD (TD9)</td><td  class='Ligne' align='right'><strong>9</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir sur table] Interrogation surprises et QCM</td><td  class='Ligne' align='right'><strong>17</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir] Exercice maison sur table (TD3)</td><td  class='Ligne' align='right'><strong>17</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir sur table] 1ère Interrogation de TD (TD5)</td><td  class='Ligne' align='right'><strong>7.5</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir] Exercice maison sur table (TD7)</td><td  class='Ligne' align='right'><strong>13.5</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[examen] Examens 1ère session</td><td  class='Ligne' align='right'><strong>9.75</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[cc-total] Note de controle continu</td><td  class='Ligne' align='right'><strong>12.19</strong>/20</tr><tr><td class='Ligne'>LI217-2013fev</td><td class='Ligne'>[examen] creation du 16-04-2013-112</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>LI217-2013fev</td><td class='Ligne'>[TP] creation du 02-05-2013-770</td><td  class='Ligne' align='right'><strong>19</strong>/20</tr><tr><td class='Ligne'>LI217-2013fev</td><td class='Ligne'>[partiel] creation du 07-02-2013-108</td><td  class='Ligne' align='right'><strong>19.5</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[partiel] Partiel (30 oct)</td><td  class='Ligne' align='right'><strong>2.5</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[devoir a la maison] Devoir Make</td><td  class='Ligne' align='right'><strong>12.7</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[devoir a la maison] Devoirs hebdomadaires</td><td  class='Ligne' align='right'><strong>5.5</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[interrogation ecrite] Commandes</td><td  class='Ligne' align='right'><strong>17.44</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[examen] Examen (15/1/2012)</td><td  class='Ligne' align='right'><strong>3.5</strong>/20</tr><tr><td class='Ligne'>LI219-2013fev</td><td class='Ligne'>[examen] Examen final</td><td  class='Ligne' align='right'><strong>13.54</strong>/20</tr><tr><td class='Ligne'>LI219-2013fev</td><td class='Ligne'>[devoir sur table] creation du 28-01-2013-309</td><td  class='Ligne' align='right'><strong>15.4</strong>/20</tr><tr><td class='Ligne'>LI219-2013fev</td><td class='Ligne'>[TME solitaire] creation du 28-01-2013-477</td><td  class='Ligne' align='right'><strong>6</strong>/20</tr><tr><td class='Ligne'>LI219-2013fev</td><td class='Ligne'>[TP] TD/TP</td><td  class='Ligne' align='right'><strong>12.9</strong>/20</tr><tr><td class='Ligne'>LI221-2012oct</td><td class='Ligne'>[TME solitaire] creation du 02-10-2012-477</td><td  class='Ligne' align='right'><strong>4.5</strong>/20</tr><tr><td class='Ligne'>LI221-2012oct</td><td class='Ligne'>[examen] creation du 02-10-2012-112</td><td  class='Ligne' align='right'><strong>26.5</strong>/60</tr><tr><td class='Ligne'>LI221-2012oct</td><td class='Ligne'>[TD] creation du 02-10-2012-2648</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>LI221-2012oct</td><td class='Ligne'>[partiel] creation du 02-10-2012-108</td><td  class='Ligne' align='right'><strong>11.5</strong>/20</tr><tr><td class='Ligne'>LI230-2012oct</td><td class='Ligne'>[examen] creation du 27-08-2012-112</td><td  class='Ligne' align='right'><strong>23.5</strong>/60</tr><tr><td class='Ligne'>LI230-2012oct</td><td class='Ligne'>[TME solitaire] creation du 27-08-2012-477</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>LI230-2012oct</td><td class='Ligne'>[TD] creation du 27-08-2012-2648</td><td  class='Ligne' align='right'><strong>13.5</strong>/20</tr><tr><td class='Ligne'>LI230-2012oct</td><td class='Ligne'>[cc-total] creation du 27-08-2012-111</td><td  class='Ligne' align='right'><strong>26.5</strong>/40</tr><tr><td class='Ligne'>LI260-2013fev</td><td class='Ligne'>[participation] creation du 24-04-2013-122</td><td  class='Ligne' align='right'><strong>11</strong>/20</tr><tr><td class='Ligne'>LI260-2013fev</td><td class='Ligne'>[examen] creation du 24-04-2013-112</td><td  class='Ligne' align='right'><strong>12.6</strong>/20</tr><tr><td class='Ligne'>LI260-2013fev</td><td class='Ligne'>[devoir] Rapport/code</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>LI260-2013fev</td><td class='Ligne'>[partiel] creation du 05-02-2013-108</td><td  class='Ligne' align='right'><strong>5.5</strong>/20</tr><tr><td class='Ligne'>LI310-2013oct</td><td class='Ligne'>[examen] examen</td><td  class='Ligne' align='right'><strong>12.25</strong>/20</tr><tr><td class='Ligne'>LI310-2013oct</td><td class='Ligne'>[partiel] partiel</td><td  class='Ligne' align='right'><strong>7.25</strong>/20</tr><tr><td class='Ligne'>LI325-2014fev</td><td class='Ligne'>[examen] creation du 07-04-2014-112</td><td  class='Ligne' align='right'><strong>11.5</strong>/20</tr><tr><td class='Ligne'>LI325-2014fev</td><td class='Ligne'>[devoir a la maison] creation du 07-04-2014-476</td><td  class='Ligne' align='right'><strong>17</strong>/20</tr><tr><td class='Ligne'>LI325-2014fev</td><td class='Ligne'>[partiel] creation du 07-04-2014-108</td><td  class='Ligne' align='right'><strong>12</strong>/20</tr><tr><td class='Ligne'>LI325-2014fev</td><td class='Ligne'>[interrogation ecrite] creation du 07-04-2014-308</td><td  class='Ligne' align='right'><strong>15.5</strong>/20</tr><tr><td class='Ligne'>LI330-2013oct</td><td class='Ligne'>[examen] creation du 13-06-2013-112</td><td  class='Ligne' align='right'><strong>7.5</strong>/20</tr><tr><td class='Ligne'>LI330-2013oct</td><td class='Ligne'>[TD] Note de TD/TME</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>LI330-2013oct</td><td class='Ligne'>[epreuve] Examen partiel</td><td  class='Ligne' align='right'><strong>6.5</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[examen] examen 1ère session -déc 2013</td><td  class='Ligne' align='right'><strong>8.85</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[devoir] devoir 2</td><td  class='Ligne' align='right'><strong>15</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[devoir] devoir 1</td><td  class='Ligne' align='right'><strong>19</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[partiel] partiel du 13/11/13</td><td  class='Ligne' align='right'><strong>4</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[cc-total] creation du 11-11-2013-111</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>LI336-2014fev</td><td class='Ligne'>[partiel] creation du 14-01-2014-108</td><td  class='Ligne' align='right'><strong>9.5</strong>/20</tr><tr><td class='Ligne'>LI336-2014fev</td><td class='Ligne'>[examen] creation du 08-04-2014-112</td><td  class='Ligne' align='right'><strong>8.5</strong>/20</tr><tr><td class='Ligne'>LI341-2013oct</td><td class='Ligne'>[examen] Examen</td><td  class='Ligne' align='right'><strong>43.25</strong>/60</tr><tr><td class='Ligne'>LI341-2013oct</td><td class='Ligne'>[partiel] creation du 10-09-2013-108</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>LI341-2013oct</td><td class='Ligne'>[interrogation ecrite] Première Interro (semaine 5)</td><td  class='Ligne' align='right'><strong>5</strong>/10</tr><tr><td class='Ligne'>LI341-2013oct</td><td class='Ligne'>[interrogation ecrite] Deuxième Interro (semaine 10)</td><td  class='Ligne' align='right'><strong>6.25</strong>/10</tr><tr><td class='Ligne'>LI342-2013oct</td><td class='Ligne'>[Epreuve TP] TME Solitaire</td><td  class='Ligne' align='right'><strong>10</strong>/20</tr><tr><td class='Ligne'>LI342-2013oct</td><td class='Ligne'>[examen] creation du 16-09-2013-112</td><td  class='Ligne' align='right'><strong>14.6</strong>/20</tr><tr><td class='Ligne'>LI342-2013oct</td><td class='Ligne'>[partiel] creation du 16-09-2013-108</td><td  class='Ligne' align='right'><strong>9.4</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[examen] Examen 1ere session</td><td  class='Ligne' align='right'><strong>10</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[interrogation ecrite] interro 2</td><td  class='Ligne' align='right'><strong>15.25</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[interrogation ecrite] Interro 1</td><td  class='Ligne' align='right'><strong>13.5</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[interrogation ecrite] interro 3</td><td  class='Ligne' align='right'><strong>8</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[Epreuve TP] Projet PHP</td><td  class='Ligne' align='right'><strong>12</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[cc-total] 2 meilleures interros + PHP</td><td  class='Ligne' align='right'><strong>13.43</strong>/20</tr><tr><td class='Ligne'>LI351-2014fev</td><td class='Ligne'>[TD] Note de TD/TME</td><td  class='Ligne' align='right'><strong>14.6</strong>/20</tr><tr><td class='Ligne'>LI351-2014fev</td><td class='Ligne'>[devoir sur table] Examen écrit du vendredi 14 mars</td><td  class='Ligne' align='right'><strong>13.75</strong>/20</tr><tr><td class='Ligne'>LI351-2014fev</td><td class='Ligne'>[examen] Examen 1ere session</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>LI351-2014fev</td><td class='Ligne'>[cc-total] Note totale de CC</td><td  class='Ligne' align='right'><strong>14.28</strong>/20</tr><tr><td class='Ligne'>LI352-2014fev</td><td class='Ligne'>[cc-total] partiel + exposé</td><td  class='Ligne' align='right'><strong>15</strong>/20</tr><tr><td class='Ligne'>LI352-2014fev</td><td class='Ligne'>[examen] 1S</td><td  class='Ligne' align='right'><strong>12.5</strong>/20</tr></table></div></p>"

it=0
a=[]
while(it<100):
    try:
        t = re.search('<strong>.*</strong>',texte)
        a.append(t.group(1))
        texte= re.search('<strong>(.*)</strong>',texte).group(1)
        it+=1
    except AttributeError:
        break


In [110]:
texte ="<tr><td class='Ligne'>4I200--2014oct</td><td class='Ligne'>[examen-reparti-1] examen-reparti-1</td><td  class='Ligne' align='right'><strong>11</strong>/20</tr><tr><td class='Ligne'>4I200--2014oct</td><td class='Ligne'>[examen] examen reparti 2</td><td  class='Ligne' align='right'><strong>8</strong>/20</tr><tr><td class='Ligne'>4I200--2014oct</td><td class='Ligne'>[epreuve] Evaluation projet optimisation equitable</td><td  class='Ligne' align='right'><strong>16</strong>/20</tr><tr><td class='Ligne'>4I502--2014oct</td><td class='Ligne'>[examen] examen réparti 1</td><td  class='Ligne' align='right'><strong>11</strong>/20</tr><tr><td class='Ligne'>4I502--2014oct</td><td class='Ligne'>[cc-total] Projet LDVH</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>4I502--2014oct</td><td class='Ligne'>[epreuve] Examen Réparti 2 : Janvier</td><td  class='Ligne' align='right'><strong>10</strong>/20</tr><tr><td class='Ligne'>4I601--2014oct</td><td class='Ligne'>[epreuve] exam de janvier</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>4I601--2014oct</td><td class='Ligne'>[epreuve] CC</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>4I601--2014oct</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 30-01-2015-533163</td><td  class='Ligne' align='right'><strong>14.3</strong>/20</tr><tr><td class='Ligne'>4I601--2014oct</td><td class='Ligne'>[examen] note dbufr 1ere session</td><td  class='Ligne' align='right'><strong>14.3</strong>/20</tr><tr><td class='Ligne'>4I800--2014oct</td><td class='Ligne'>[examen-reparti-2] creation du 19-01-2015-533162</td><td  class='Ligne' align='right'><strong>7.5</strong>/20</tr><tr><td class='Ligne'>4I800--2014oct</td><td class='Ligne'>[examen] creation du 06-12-2014-112</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>4I801--2014oct</td><td class='Ligne'>[examen-reparti-1] 5 novembre 2014</td><td  class='Ligne' align='right'><strong>17</strong>/20</tr><tr><td class='Ligne'>4I801--2014oct</td><td class='Ligne'>[examen-reparti-2] janvier 2015</td><td  class='Ligne' align='right'><strong>16</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[epreuve] projet 1</td><td  class='Ligne' align='right'><strong>15</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[epreuve] Projet 2</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[examen-reparti-2] examen</td><td  class='Ligne' align='right'><strong>11</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 03-06-2015-533163</td><td  class='Ligne' align='right'><strong>12.4</strong>/20</tr><tr><td class='Ligne'>4I802--2015fev</td><td class='Ligne'>[examen-reparti-1] creation du 03-06-2015-533161</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>4I803--2015fev</td><td class='Ligne'>[examen-reparti-1] examen-reparti-1</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>4I803--2015fev</td><td class='Ligne'>[examen-reparti-2] 19 mai 2015</td><td  class='Ligne' align='right'><strong>12</strong>/20</tr><tr><td class='Ligne'>4I803--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] note finale session1</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>4I804--2015fev</td><td class='Ligne'>[examen] creation du 05-06-2015-112</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>4I804--2015fev</td><td class='Ligne'>[examen-reparti-1] creation du 05-06-2015-533161</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>4I804--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 05-06-2015-533163</td><td  class='Ligne' align='right'><strong>13.4</strong>/20</tr><tr><td class='Ligne'>4I807--2015fev</td><td class='Ligne'>[epreuve] Recherche Bibliographique</td><td  class='Ligne' align='right'><strong>18</strong>/20</tr><tr><td class='Ligne'>4I807--2015fev</td><td class='Ligne'>[epreuve] Rapport + Soutenance</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>4I807--2015fev</td><td class='Ligne'>[epreuve] Projet</td><td  class='Ligne' align='right'><strong>12</strong>/20</tr><tr><td class='Ligne'>4I807--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 04-06-2015-533163</td><td  class='Ligne' align='right'><strong>12.65</strong>/20</tr><tr><td class='Ligne'>4I808--2015fev</td><td class='Ligne'>[examen-reparti-1] Projet (rapport, soutenance, application)</td><td  class='Ligne' align='right'><strong>12.5</strong>/20</tr><tr><td class='Ligne'>4I808--2015fev</td><td class='Ligne'>[examen-reparti-2] Examen de fin de semestre</td><td  class='Ligne' align='right'><strong>16.45</strong>/20</tr><tr><td class='Ligne'>4I808--2015fev</td><td class='Ligne'>[Note finale DBUFR premiere session] creation du 02-06-2015-533163</td><td  class='Ligne' align='right'><strong>14.08</strong>/20</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[TME solitaire] creation du 10-02-2012-477</td><td  class='Ligne' align='right'><strong>18.75</strong>/20</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[TD] creation du 10-02-2012-2648</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[partiel] creation du 10-02-2012-108</td><td  class='Ligne' align='right'><strong>8</strong>/20</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[examen] creation du 10-02-2012-112</td><td  class='Ligne' align='right'><strong>40.75</strong>/70</tr><tr><td class='Ligne'>LI115-MIME-2012fev</td><td class='Ligne'>[cc-total] creation du 10-02-2012-111</td><td  class='Ligne' align='right'><strong>12.4</strong>/20</tr><tr><td class='Ligne'>LI213-2013fev</td><td class='Ligne'>[epreuve] Epreuve écrite 1ere session</td><td  class='Ligne' align='right'><strong>13.25</strong>/20</tr><tr><td class='Ligne'>LI213-2013fev</td><td class='Ligne'>[Epreuve TP] TME1-2</td><td  class='Ligne' align='right'><strong>8</strong>/20</tr><tr><td class='Ligne'>LI213-2013fev</td><td class='Ligne'>[TD] Note TD (interros+participation)</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>LI213-2013fev</td><td class='Ligne'>[Epreuve TP] Projet</td><td  class='Ligne' align='right'><strong>15</strong>/20</tr><tr><td class='Ligne'>LI215-2012oct</td><td class='Ligne'>[participation] TP et Participation</td><td  class='Ligne' align='right'><strong>16.75</strong>/20</tr><tr><td class='Ligne'>LI215-2012oct</td><td class='Ligne'>[examen] Ecrit première Session</td><td  class='Ligne' align='right'><strong>19.5</strong>/60</tr><tr><td class='Ligne'>LI215-2012oct</td><td class='Ligne'>[cc-total] creation du 16-09-2012-111</td><td  class='Ligne' align='right'><strong>24.25</strong>/40</tr><tr><td class='Ligne'>LI215-2012oct</td><td class='Ligne'>[partiel] Partiel barême ajusté</td><td  class='Ligne' align='right'><strong>7.5</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir sur table] 2ième interrogation de TD (TD9)</td><td  class='Ligne' align='right'><strong>9</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir sur table] Interrogation surprises et QCM</td><td  class='Ligne' align='right'><strong>17</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir] Exercice maison sur table (TD3)</td><td  class='Ligne' align='right'><strong>17</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir sur table] 1ère Interrogation de TD (TD5)</td><td  class='Ligne' align='right'><strong>7.5</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[devoir] Exercice maison sur table (TD7)</td><td  class='Ligne' align='right'><strong>13.5</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[examen] Examens 1ère session</td><td  class='Ligne' align='right'><strong>9.75</strong>/20</tr><tr><td class='Ligne'>LI216-2013fev</td><td class='Ligne'>[cc-total] Note de controle continu</td><td  class='Ligne' align='right'><strong>12.19</strong>/20</tr><tr><td class='Ligne'>LI217-2013fev</td><td class='Ligne'>[examen] creation du 16-04-2013-112</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>LI217-2013fev</td><td class='Ligne'>[TP] creation du 02-05-2013-770</td><td  class='Ligne' align='right'><strong>19</strong>/20</tr><tr><td class='Ligne'>LI217-2013fev</td><td class='Ligne'>[partiel] creation du 07-02-2013-108</td><td  class='Ligne' align='right'><strong>19.5</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[partiel] Partiel (30 oct)</td><td  class='Ligne' align='right'><strong>2.5</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[devoir a la maison] Devoir Make</td><td  class='Ligne' align='right'><strong>12.7</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[devoir a la maison] Devoirs hebdomadaires</td><td  class='Ligne' align='right'><strong>5.5</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[interrogation ecrite] Commandes</td><td  class='Ligne' align='right'><strong>17.44</strong>/20</tr><tr><td class='Ligne'>LI218-2012oct</td><td class='Ligne'>[examen] Examen (15/1/2012)</td><td  class='Ligne' align='right'><strong>3.5</strong>/20</tr><tr><td class='Ligne'>LI219-2013fev</td><td class='Ligne'>[examen] Examen final</td><td  class='Ligne' align='right'><strong>13.54</strong>/20</tr><tr><td class='Ligne'>LI219-2013fev</td><td class='Ligne'>[devoir sur table] creation du 28-01-2013-309</td><td  class='Ligne' align='right'><strong>15.4</strong>/20</tr><tr><td class='Ligne'>LI219-2013fev</td><td class='Ligne'>[TME solitaire] creation du 28-01-2013-477</td><td  class='Ligne' align='right'><strong>6</strong>/20</tr><tr><td class='Ligne'>LI219-2013fev</td><td class='Ligne'>[TP] TD/TP</td><td  class='Ligne' align='right'><strong>12.9</strong>/20</tr><tr><td class='Ligne'>LI221-2012oct</td><td class='Ligne'>[TME solitaire] creation du 02-10-2012-477</td><td  class='Ligne' align='right'><strong>4.5</strong>/20</tr><tr><td class='Ligne'>LI221-2012oct</td><td class='Ligne'>[examen] creation du 02-10-2012-112</td><td  class='Ligne' align='right'><strong>26.5</strong>/60</tr><tr><td class='Ligne'>LI221-2012oct</td><td class='Ligne'>[TD] creation du 02-10-2012-2648</td><td  class='Ligne' align='right'><strong>14</strong>/20</tr><tr><td class='Ligne'>LI221-2012oct</td><td class='Ligne'>[partiel] creation du 02-10-2012-108</td><td  class='Ligne' align='right'><strong>11.5</strong>/20</tr><tr><td class='Ligne'>LI230-2012oct</td><td class='Ligne'>[examen] creation du 27-08-2012-112</td><td  class='Ligne' align='right'><strong>23.5</strong>/60</tr><tr><td class='Ligne'>LI230-2012oct</td><td class='Ligne'>[TME solitaire] creation du 27-08-2012-477</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>LI230-2012oct</td><td class='Ligne'>[TD] creation du 27-08-2012-2648</td><td  class='Ligne' align='right'><strong>13.5</strong>/20</tr><tr><td class='Ligne'>LI230-2012oct</td><td class='Ligne'>[cc-total] creation du 27-08-2012-111</td><td  class='Ligne' align='right'><strong>26.5</strong>/40</tr><tr><td class='Ligne'>LI260-2013fev</td><td class='Ligne'>[participation] creation du 24-04-2013-122</td><td  class='Ligne' align='right'><strong>11</strong>/20</tr><tr><td class='Ligne'>LI260-2013fev</td><td class='Ligne'>[examen] creation du 24-04-2013-112</td><td  class='Ligne' align='right'><strong>12.6</strong>/20</tr><tr><td class='Ligne'>LI260-2013fev</td><td class='Ligne'>[devoir] Rapport/code</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>LI260-2013fev</td><td class='Ligne'>[partiel] creation du 05-02-2013-108</td><td  class='Ligne' align='right'><strong>5.5</strong>/20</tr><tr><td class='Ligne'>LI310-2013oct</td><td class='Ligne'>[examen] examen</td><td  class='Ligne' align='right'><strong>12.25</strong>/20</tr><tr><td class='Ligne'>LI310-2013oct</td><td class='Ligne'>[partiel] partiel</td><td  class='Ligne' align='right'><strong>7.25</strong>/20</tr><tr><td class='Ligne'>LI325-2014fev</td><td class='Ligne'>[examen] creation du 07-04-2014-112</td><td  class='Ligne' align='right'><strong>11.5</strong>/20</tr><tr><td class='Ligne'>LI325-2014fev</td><td class='Ligne'>[devoir a la maison] creation du 07-04-2014-476</td><td  class='Ligne' align='right'><strong>17</strong>/20</tr><tr><td class='Ligne'>LI325-2014fev</td><td class='Ligne'>[partiel] creation du 07-04-2014-108</td><td  class='Ligne' align='right'><strong>12</strong>/20</tr><tr><td class='Ligne'>LI325-2014fev</td><td class='Ligne'>[interrogation ecrite] creation du 07-04-2014-308</td><td  class='Ligne' align='right'><strong>15.5</strong>/20</tr><tr><td class='Ligne'>LI330-2013oct</td><td class='Ligne'>[examen] creation du 13-06-2013-112</td><td  class='Ligne' align='right'><strong>7.5</strong>/20</tr><tr><td class='Ligne'>LI330-2013oct</td><td class='Ligne'>[TD] Note de TD/TME</td><td  class='Ligne' align='right'><strong>13</strong>/20</tr><tr><td class='Ligne'>LI330-2013oct</td><td class='Ligne'>[epreuve] Examen partiel</td><td  class='Ligne' align='right'><strong>6.5</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[examen] examen 1ère session -déc 2013</td><td  class='Ligne' align='right'><strong>8.85</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[devoir] devoir 2</td><td  class='Ligne' align='right'><strong>15</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[devoir] devoir 1</td><td  class='Ligne' align='right'><strong>19</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[partiel] partiel du 13/11/13</td><td  class='Ligne' align='right'><strong>4</strong>/20</tr><tr><td class='Ligne'>LI332-2013oct</td><td class='Ligne'>[cc-total] creation du 11-11-2013-111</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>LI336-2014fev</td><td class='Ligne'>[partiel] creation du 14-01-2014-108</td><td  class='Ligne' align='right'><strong>9.5</strong>/20</tr><tr><td class='Ligne'>LI336-2014fev</td><td class='Ligne'>[examen] creation du 08-04-2014-112</td><td  class='Ligne' align='right'><strong>8.5</strong>/20</tr><tr><td class='Ligne'>LI341-2013oct</td><td class='Ligne'>[examen] Examen</td><td  class='Ligne' align='right'><strong>43.25</strong>/60</tr><tr><td class='Ligne'>LI341-2013oct</td><td class='Ligne'>[partiel] creation du 10-09-2013-108</td><td  class='Ligne' align='right'><strong>14.5</strong>/20</tr><tr><td class='Ligne'>LI341-2013oct</td><td class='Ligne'>[interrogation ecrite] Première Interro (semaine 5)</td><td  class='Ligne' align='right'><strong>5</strong>/10</tr><tr><td class='Ligne'>LI341-2013oct</td><td class='Ligne'>[interrogation ecrite] Deuxième Interro (semaine 10)</td><td  class='Ligne' align='right'><strong>6.25</strong>/10</tr><tr><td class='Ligne'>LI342-2013oct</td><td class='Ligne'>[Epreuve TP] TME Solitaire</td><td  class='Ligne' align='right'><strong>10</strong>/20</tr><tr><td class='Ligne'>LI342-2013oct</td><td class='Ligne'>[examen] creation du 16-09-2013-112</td><td  class='Ligne' align='right'><strong>14.6</strong>/20</tr><tr><td class='Ligne'>LI342-2013oct</td><td class='Ligne'>[partiel] creation du 16-09-2013-108</td><td  class='Ligne' align='right'><strong>9.4</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[examen] Examen 1ere session</td><td  class='Ligne' align='right'><strong>10</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[interrogation ecrite] interro 2</td><td  class='Ligne' align='right'><strong>15.25</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[interrogation ecrite] Interro 1</td><td  class='Ligne' align='right'><strong>13.5</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[interrogation ecrite] interro 3</td><td  class='Ligne' align='right'><strong>8</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[Epreuve TP] Projet PHP</td><td  class='Ligne' align='right'><strong>12</strong>/20</tr><tr><td class='Ligne'>LI345-2014fev</td><td class='Ligne'>[cc-total] 2 meilleures interros + PHP</td><td  class='Ligne' align='right'><strong>13.43</strong>/20</tr><tr><td class='Ligne'>LI351-2014fev</td><td class='Ligne'>[TD] Note de TD/TME</td><td  class='Ligne' align='right'><strong>14.6</strong>/20</tr><tr><td class='Ligne'>LI351-2014fev</td><td class='Ligne'>[devoir sur table] Examen écrit du vendredi 14 mars</td><td  class='Ligne' align='right'><strong>13.75</strong>/20</tr><tr><td class='Ligne'>LI351-2014fev</td><td class='Ligne'>[examen] Examen 1ere session</td><td  class='Ligne' align='right'><strong>10.5</strong>/20</tr><tr><td class='Ligne'>LI351-2014fev</td><td class='Ligne'>[cc-total] Note totale de CC</td><td  class='Ligne' align='right'><strong>14.28</strong>/20</tr><tr><td class='Ligne'>LI352-2014fev</td><td class='Ligne'>[cc-total] partiel + exposé</td><td  class='Ligne' align='right'><strong>15</strong>/20</tr><tr><td class='Ligne'>LI352-2014fev</td><td class='Ligne'>[examen] 1S</td><td  class='Ligne' align='right'><strong>12.5</strong>/20</tr></table></div></p>"
notes = np.array([float(n) for n in re.findall('<strong>(.*?)</strong>',texte)])
sur=np.array([float(s) for s in re.findall('</strong>/(.*?)</tr>',texte)])
((notes/sur)*20).mean()

12.148749463749464